<a href="https://colab.research.google.com/github/Gk58IISERTvm98/Plotting_and_Visualization/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import numpy as np
import string

In [2]:
train_data = pd.read_csv("/content/Titanic_Passanger_Survivals/Titanic_data/train.csv")
test_data = pd.read_csv("/content/Titanic_Passanger_Survivals/Titanic_data/test.csv")

CombData = [train_data, test_data]

In [3]:
for dataset in CombData:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)


In [4]:
for dataset in CombData:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [5]:
train_data[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

,Title,Survived
0,Master,0.575000
1,Miss,0.702703
2,Mr,0.156673
3,Mrs,0.793651
4,Rare,0.347826


In [6]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in CombData:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [7]:
for dataset in CombData:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

# Adding New Feature

In [8]:
for dataset in CombData:
  tp = []
  for name in dataset.Name:
    title = re.search(' ([A-Za-z]+)\.', name)
    name = name.replace(title.group(0),"")
    punct=str.maketrans('','',string.punctuation)
    name=name.translate(punct)
    tp.append(len(name.split()))
  dataset['Total_Membors'] = tp

In [9]:
train_data[['Total_Membors', 'Survived']].groupby(['Total_Membors'], as_index=False).mean()

,Total_Membors,Survived
0,2,0.291803
1,3,0.341902
2,4,0.427083
3,5,0.769231
4,6,0.842105
5,7,0.555556
6,8,1.000000
7,13,1.000000


In [10]:
for dataset in CombData:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

for dataset in CombData:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

for dataset in CombData:
    dataset['relatives'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['relatives'] > 0, 'IsAlone'] = 0
    dataset.loc[dataset['relatives'] == 0, 'IsAlone'] = 1
    dataset['IsAlone'] = dataset['IsAlone'].astype(int)

In [11]:
for dataset in CombData:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

In [12]:
for dataset in CombData:
  dataset['FarePerMember'] = dataset['Fare']/dataset['Total_Membors']

In [13]:
train_data = train_data.drop(['Name', 'PassengerId', 'Ticket','Cabin'], axis=1)
test_data = test_data.drop(['Name','Ticket','Cabin'], axis=1)
CombData = [train_data, test_data]

# Filling Missing Values
**Ebmarked**

In [14]:
freq_port = train_data.Embarked.dropna().mode()[0]
for dataset in CombData:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

**filling Missing Values of Age Column**

**METHOD First**



```
guess_ages = np.zeros((2,3))
guess_ages

for dataset in CombData:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)
# print(train_data['Age'][20:60])
```



**METHOD 02**



```
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

missingdata1 = IterativeImputer(random_state=0)
missingdata1.fit(train_data)
dd = missingdata1.transform(train_data).astype(int)
train = pd.DataFrame(dd)
train.columns = train_data.columns
  
# print(train['Age'][60:100])

missingdata1 = IterativeImputer(random_state=0)
missingdata1.fit(test_data)
msd = missingdata1.transform(test_data).astype(int)
test = pd.DataFrame(msd)
test.columns = test_data.columns

combine = [train, test]
```



**METHOD 03**


```
for dataset in CombData:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
```



**METHOD 04**



```
for dataset in CombData:
  age_nan_indices = list(dataset[dataset['Age'].isnull()].index)
  len(age_nan_indices)
  for index in age_nan_indices:
      median_age = dataset['Age'].median()
      predict_age = dataset['Age'][(dataset['SibSp'] == dataset.iloc[index]['SibSp'])
                                  & (dataset['Parch'] == dataset.iloc[index]['Parch'])
                                  & (dataset['Pclass'] == dataset.iloc[index]["Pclass"])].median()
      if np.isnan(predict_age):
          dataset['Age'].iloc[index] = median_age
      else:
          dataset['Age'].iloc[index] = predict_age
```



In [15]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

train_data = pd.get_dummies(train_data)
missingdata1 = IterativeImputer(random_state=0)
missingdata1.fit(train_data)
dd = missingdata1.transform(train_data).astype(int)
train = pd.DataFrame(dd)
train.columns = train_data.columns

# print(train['Age'][60:100])
test_data = pd.get_dummies(test_data)
missingdata1 = IterativeImputer(random_state=0)
missingdata1.fit(test_data)
msd = missingdata1.transform(test_data).astype(int)
test = pd.DataFrame(msd)
test.columns = test_data.columns

combine = [train, test]

/usr/local/lib/python3.10/dist-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [16]:
train['Age_Band'] = pd.cut(train_data['Age'].to_numpy(), 5)
train[['Age_Band', 'Survived']].groupby(['Age_Band'], as_index=False).mean().sort_values(by = 'Age_Band')

<ipython-input-16-44a87ff15dee>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train[['Age_Band', 'Survived']].groupby(['Age_Band'], as_index=False).mean().sort_values(by = 'Age_Band')


,Age_Band,Survived
0,"(0.34, 16.336]",0.550000
1,"(16.336, 32.252]",0.369942
2,"(32.252, 48.168]",0.404255
3,"(48.168, 64.084]",0.434783
4,"(64.084, 80.0]",0.090909


In [17]:
for dataset in combine:
    dataset.loc[ dataset['Age'] <= 16, 'AgeOneHot'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'AgeOneHot'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'AgeOneHot'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'AgeOneHot'] = 3
    dataset.loc[ dataset['Age'] > 64, 'AgeOneHot'] = 4
    dataset['AgeOneHot'] = dataset['AgeOneHot'].astype(int)

**One Hot Encoding of Fair Column**



```
train_data['FareBand'] = pd.cut(train_data['Fare'], 4)
train_data[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by = 'FareBand')    
for dataset in CombData:
  dataset['Fare'] = dataset['Fare'].map(lambda x: np.log(x) if x > 0 else 0)
  dataset.loc[combine['Fare'] <= 1.56, 'Fare'] = 0
  dataset.loc[(combine['Fare'] > 1.56) & (dataset['Fare'] <= 3.119), 'Fare'] = 1
  dataset.loc[(combine['Fare'] > 3.119) & (dataset['Fare'] <= 4.679), 'Fare'] = 2
  dataset.loc[combine['Fare'] > 4.679, 'Fare'] = 3
```



In [18]:
train['FareBand'] = pd.cut(train_data['Fare'], 4)
train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by = 'FareBand')

<ipython-input-18-223987dd4ac4>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by = 'FareBand')


,FareBand,Survived
0,"(-0.512, 128.082]",0.368113
1,"(128.082, 256.165]",0.724138
2,"(256.165, 384.247]",0.666667
3,"(384.247, 512.329]",1.000000


In [19]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'FareOnehot'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'FareOnehot'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'FareOnehot']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'FareOnehot'] = 3
    dataset['FareOnehot'] = dataset['FareOnehot'].astype(int)

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   Survived       891 non-null    int64   
 1   Pclass         891 non-null    int64   
 2   Sex            891 non-null    int64   
 3   Age            891 non-null    int64   
 4   SibSp          891 non-null    int64   
 5   Parch          891 non-null    int64   
 6   Fare           891 non-null    int64   
 7   Embarked       891 non-null    int64   
 8   Title          891 non-null    int64   
 9   Total_Membors  891 non-null    int64   
 10  FamilySize     891 non-null    int64   
 11  IsAlone        891 non-null    int64   
 12  relatives      891 non-null    int64   
 13  Age*Class      891 non-null    int64   
 14  FarePerMember  891 non-null    int64   
 15  Age_Band       714 non-null    category
 16  AgeOneHot      891 non-null    int64   
 17  FareBand       891 non-null    cate